# APMA E4990 Final Project: Wine Master

Xiaojing Dong, Keran Li, Zhen Li, Zihan Yi  
Webapp: http://winemaster.ml or http://winemaster.pythonanywhere.com

## Project Goal

Motivation: 
- To access information on a specific wine without typing 
- To choose the right wine without being a wine expert

## Web Development

![](./Final Presentation/Project Goal.png)

- http://winemaster.ml or http://winemaster.pythonanywhere.com


- Develop and host the Flask Webapp with PythonAnywhere
    - Design the homepage and app page with html and css


- Create a SQL database with the PythonAnywhere online MySQL
    - Connect the database to our flask app, use it to store and retrieve data


- Load the OCR, text matching and recommendation models to the webapp

## Data Gathering and Cleansing

#### 1. Database: Wine Review data from Kaggle: https://www.kaggle.com/zynicide/wine-reviews


i. Remove countries with frequency less than 1000: remain 12 countries (using 7 languages) to reduce the cost of OCR  
ii. Extract column *year* from column *title*  
iii. Remove redundant column *region_2* which contains same information as *region_1* does  
iv. Remove useless columns *taster_twitter_handle* and *taster_name*
v. Merge duplicate rows for same wines:  
- compute the average of non-NA "points"  
- compute the average of non-NA "price"  
- concate the text information of "description"  

In [1]:
import pandas as pd
wine = pd.read_csv('./Data/wine_data_cleaned.csv')
wine.shape

(116759, 13)

In [2]:
wine.head()

,index,country,province,region_1,designation,variety,winery,year,title,unique_name,description,points,price
0,0.0,Italy,Sicily & Sardinia,Etna,Vulkà Bianco,White Blend,Nicosia,2013.0,Nicosia 2013 Vulkà Bianco (Etna),Nicosia 2013 Vulkà Bianco (Etna) White Blend,"Aromas include tropical fruit, broom, brimston...",87.0,NaN
1,1.0,Portugal,Douro,NaN,Avidagos,Portuguese Red,Quinta dos Avidagos,2011.0,Quinta dos Avidagos 2011 Avidagos Red (Douro),Quinta dos Avidagos 2011 Avidagos Red (Douro) ...,"This is ripe and fruity, a wine that is smooth...",87.0,15.0
2,2.0,US,Oregon,Willamette Valley,NaN,Pinot Gris,Rainstorm,2013.0,Rainstorm 2013 Pinot Gris (Willamette Valley),Rainstorm 2013 Pinot Gris (Willamette Valley) ...,"Tart and snappy, the flavors of lime flesh and...",87.0,14.0
3,3.0,US,Michigan,Lake Michigan Shore,Reserve Late Harvest,Riesling,St. Julian,2013.0,St. Julian 2013 Reserve Late Harvest Riesling ...,St. Julian 2013 Reserve Late Harvest Riesling ...,"Pineapple rind, lemon pith and orange blossom ...",87.0,13.0
4,4.0,US,Oregon,Willamette Valley,Vintner's Reserve Wild Child Block,Pinot Noir,Sweet Cheeks,2012.0,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,"Much like the regular bottling from 2012, this...",87.0,65.0


#### 2. 120 wine labels scraped from https://www.wine.com

- Web scraping codes: https://github.com/zy2292/Wine-Master/blob/master/Codes/data_gathering_and_preparation/web_scraping.ipynb

- Manually match the wine labels against entries in our database, store the results in *training_label.csv*

- Randomly split the labels into two sets
    - 100 training labels
    - 20 testing labels

## Part I. Wine Label Recognition with Tesseract OCR
### Image preprocessing to improve OCR performance

- Convert images to grayscale  
- Text detection with Connectionist Text Proposal Network (CTPN): modify a pre-trained CTPN (https://github.com/eragonruan/text-detection-ctpn) to return the location of text boxes

![](./Final Presentation/CTPN.png)

We also tried bilatral filter (an edge-preserving and noise-reducing smoothing filter) and canny edge detection. However, the overall performance get worse with them, probability because of the line and word finding algorithm that the Tesseract OCR engine uses.  
(https://github.com/tesseract-ocr/docs/blob/master/tesseracticdar2007.pdf)

![](./Final Presentation/OCR.png)

In [3]:
##################################################################
######## Use the pre-trained CTPN which has been modified ########
########       and store the location of text boxes       ########
##################################################################
# import ocrfunction 

# boxes = dict()
# for i in range(1,101):
#     Num = i
#     img = cv2.imread('../Data/Image/train_'+ str(num) +'.jpg')
#     boxes[i] = ocrfunction.ocr(img)

# with open('../Data/text_detection_boxes.json', 'w') as fp:
#     json.dump(boxes, fp)

### Read training wine labels with Tesseract OCR

In [4]:
import numpy as np
import json
import string
import pytesseract
import cv2
from PIL import Image
from os.path import isfile, join
from datetime import datetime

Load the location of text boxes

In [13]:
with open('./Data/text_detection_boxes.json') as f:
    boxes = json.load(f)

For each training image, combine the OCR results extracted from different text boxes

In [15]:
def text_ocr(boxes):
    result = dict()
    for i in list(range(len(boxes))):
        num = i+1
        img = cv2.imread('./Data/Image/train_'+ str(num) +'.jpg', 0)
        res = ''
        for j in list(range(len(boxes[str(i+1)]))):
            location = list(map(int,boxes[str(i+1)][str(j+1)].split(',')))
            min_x,min_y,max_x,max_y = location[0],location[1],location[2],location[3]
            image = img[min_y:max_y,min_x:max_x]
            text = pytesseract.image_to_string(image, config = '--psm 3', lang = 'eng+fra+deu+ita+spa+por+afr')
            res = res+text
        result[i]=res
    return result

In [19]:
text = text_ocr(boxes)

In [16]:
text_str = pd.DataFrame(text,index=[0]).T
text_str.head()

,0
0,MUMM NAPABRUT: POR EST LE
1,"poe ition,\n*\n\n4 €\nae EnBLANC"
2,AND BO]ER\nO\nemo DU\nCT OF FRANCE _”> CHAMPAG...
3,BRUT ROSEa ER
4,JOSEPH PHELPSREDeWINE[APA VALLEYESTATE GROWN


## Part II.  Text Matching using Tversky index

Exploratory findings:  
Generally, a wine label would have the following information which can uniquely match a wine in our database.  
- province
- region_1
- designation
- variety
- winery
- year

We therefore decide to match the OCR strings using these 6 columns.

### String preprocessing

i. Remove the non-alphanumeric characters  
ii. Remove whitespace and line breaks  
iii. Convert all characters to uppercase

In [5]:
def str_process(string):
    string=string.replace('\n','')
    string=string.replace(' ', '')
    string=''.join([*filter(str.isalnum, string)])
    string=''.join(string)
    string=string.upper()
    return string

OCR results

In [18]:
text_str[0] = text_str[0].apply(str_process)
text_str.head()

,0
0,MUMMNAPABRUTPORESTLE
1,POEITION4AEENBLANC
2,ANDBOEROEMODUCTOFFRANCECHAMPAGNEPERRIERJOUET7E...
3,BRUTROSEAER
4,JOSEPHPHELPSREDEWINEAPAVALLEYESTATEGROWN


Columns in our database to match against

In [6]:
wine['designation']=wine['designation'].apply(str).apply(str_process)
wine['province']=wine['province'].apply(str).apply(str_process)
wine['region_1']=wine['region_1'].apply(str).apply(str_process)
wine['variety']=wine['variety'].apply(str).apply(str_process)
wine['winery']=wine['winery'].apply(str).apply(str_process)
wine['year']=wine['year'].fillna(0).apply(int).apply(str).apply(str_process)

### Implement approximate string matching algorithm

#### Tokenizer: n-gram
- Treat the input string as a set of n consecutive characters. Example: 
    - "MUMMNAPA" 
    - {"MUM", "UMM", "MMN", "MNA", "NAP", "APA"}
- Consider using trigram or fourgram models

#### Similarity measure: Tversky index  
- The Tversky index for sets X and Y is a similarity score in the range [0,1]
\begin{equation}
S(X,Y)=\frac{|X\cap Y|}{|X\cap Y|+\alpha|X-Y|+\beta|Y-X|}
\end{equation}
where $\alpha$ and $\beta$ correspond to the weight of X and Y

#### Use weighted average of Tversky indices as the overall similarity score
- Motivation: Given different importance to each column 
    - For example, if the wine label has *year* specified, it should be given a larger weight in order to identify the correct wine.
    - However, the resulting Tversky index does not vary much if we change the $\alpha$ and $\beta$.
    - Consider using the weighted average of Tversky indices and set $\alpha = \beta = 0.5$


- Compute 6 Tversky indices one for each column  
- Compute the weighted average of the Tversky indices

In [7]:
from py_stringmatching.tokenizer.qgram_tokenizer import QgramTokenizer
from py_stringmatching.similarity_measure.tversky_index import TverskyIndex
from multiprocessing import Pool
from functools import partial

First, write function to compute Tversky index for each column

In [8]:
def get_tversky_index(proto, query, n=3, beta=0.5):
    tversky = TverskyIndex(beta=beta)
    qgram = QgramTokenizer(qval=n, padding=False)
    inters = tversky.get_sim_score(qgram.tokenize(query), qgram.tokenize(proto))
    return inters

def match_col(arg, query, n=3):
    col, beta = arg
    table = wine[col]
    score = table.apply(lambda x: get_tversky_index(x, query, n=n, beta=beta))
    return score

Parallelize the computation process of 6 Tversky indices

In [9]:
def get_weighted_index(query, weights, n):
    cols = ['designation', 'province','region_1','variety','winery','year']
    betas = [0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
    args = zip(cols, betas)
    
    pool = Pool()
    func = partial(match_col, query=query, n=n)
    score = pool.map(func, args)
    wine['scores'] = np.average(score, axis=0, weights=weights)
    result = wine[wine['scores']==max(wine['scores'])]
    
    if sum(result['scores'])==0:
        res_index=[]
    else:
        if result.shape[0] > 8:
            result = result.sample(8)
        res_index=list(result.index)
        
    pool.close()
    pool.join()
    return res_index

def get_match(x, y):
    return int(len(x.intersection(y)) != 0)
vec_match = np.vectorize(get_match)

For each training wine label, *training_label.csv* stores the indices that corresponds to the correct wine(s).  
Then we could select the optimal weights in a "supervised" way.

In [10]:
labeldf = pd.read_csv('./Data/training_label.csv')
labeldf = labeldf.iloc[0:100,:]
label = labeldf.iloc[:,1]
label_set = label.apply(lambda x: set(list(map(int,x.split(',')))))
label_set.head()

0                               {42190, 16919}
1    {73926, 8461, 29905, 62291, 16918, 30040}
2                                      {41747}
3                               {30024, 84417}
4                                      {87400}
Name: unique_name, dtype: object

Calculate the number of wines that are correctly matched

In [12]:
def select_weighted_matching(text_str, label_set, weights, n):
    result = text_str[0].apply(lambda x: get_weighted_index(x,weights,n)).apply(set)
    match = vec_match(result, label_set)
    return sum(match)

### Model Selection

Select the optimal weights when calculating the weighted Tversky index
- Increase the weight of *year*  
    - If the wine label has *year* information, it should be matched exactly to identify the correct wine.
- Adjust the relative weights of *province* and *region_1*
    - Information are largely overlapped.


![](./Final Presentation/Matching.png)

Therefore, we choose the optimal model using
- Tesseract OCR with CTPN text detection, and
- weighted Tversky index with 
    - weight of *year* = 1.6, 
    - weight of *region_1* = 1.3,
    - weight of *province* = 0.1, and
    - other weights being 1.0

In [15]:
with open('./Data/test_text_detection_boxes.json') as f:
    test_boxes = json.load(f)

def test_text_ocr(boxes):
    result = dict()
    for i in list(range(len(boxes))):
        num = i+1
        img = cv2.imread('./Data/Image/test_'+ str(num) +'.jpg', 0)
        res = ''
        for j in list(range(len(boxes[str(i+1)]))):
            location = list(map(int,boxes[str(i+1)][str(j+1)].split(',')))
            min_x,min_y,max_x,max_y = location[0],location[1],location[2],location[3]
            image = img[min_y:max_y,min_x:max_x]
            text = pytesseract.image_to_string(image, config = '--psm 3', lang = 'eng+fra+deu+ita+spa+por+afr')
            res = res+text
        result[i]=res
    return result

In [16]:
test_text = test_text_ocr(test_boxes)

In [18]:
test_text_str = pd.DataFrame(test_text,index=[0]).T
test_text_str[0] = test_text_str[0].apply(str_process)
test_text_str.head()

,0
0,MONTEREYCOUNTYFAMILYOWNEDAAAPALOVINTAGE
1,EBERLEESTATEVINEYARDEB
2,SONOMAMOUNTAINLINNODCABERNETSAUVIGNONAEVINEYAR...
3,VINEYARDSARROYOGRANDEVALLEYCHARDONNAYALCOHOL13...
4,SLHESTATEGROWNPRODUCEDSANTALUCIAHIGHLANDSCHARD...


In [19]:
test_labeldf = pd.read_csv('./Data/training_label.csv')
test_labeldf = test_labeldf.iloc[100:120,:]
test_label = test_labeldf.iloc[:,1]
test_label_set = test_label.apply(lambda x: set(list(map(int,x.split(',')))))
test_label_set.head()

100     {2188, 74836}
101           {10241}
102    {39864, 74840}
103           {60107}
104           {92049}
Name: unique_name, dtype: object

In [20]:
select_weighted_matching(test_text_str, test_label_set, weights=[1, 0.1, 1.3, 1, 1, 1.6], n=4)

9

The correct matching rate for 20 test labels is 9/20 = 45%.

## Part III. Content-based Recommendation

- Cold start  
- No user data

Therefore, we will use wine *desciption* to do content-based recommendation.

### String preprocessing

i. Replace '-' with whitespace  
ii. Remove the non-alphanumeric characters  
iii. Remove the stop words in English  
iv. Convert all characters to lowercase  
v. Stem the words

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from nltk.tokenize import word_tokenize
from nltk.util import ngrams
from itertools import chain
import nltk.stem

In [31]:
wine = pd.read_csv('../Data/wine_data_cleaned.csv')
wine.head()

,index,country,province,region_1,designation,variety,winery,year,title,unique_name,description,points,price
0,0.0,Italy,Sicily & Sardinia,Etna,Vulkà Bianco,White Blend,Nicosia,2013.0,Nicosia 2013 Vulkà Bianco (Etna),Nicosia 2013 Vulkà Bianco (Etna) White Blend,"Aromas include tropical fruit, broom, brimston...",87.0,NaN
1,1.0,Portugal,Douro,NaN,Avidagos,Portuguese Red,Quinta dos Avidagos,2011.0,Quinta dos Avidagos 2011 Avidagos Red (Douro),Quinta dos Avidagos 2011 Avidagos Red (Douro) ...,"This is ripe and fruity, a wine that is smooth...",87.0,15.0
2,2.0,US,Oregon,Willamette Valley,NaN,Pinot Gris,Rainstorm,2013.0,Rainstorm 2013 Pinot Gris (Willamette Valley),Rainstorm 2013 Pinot Gris (Willamette Valley) ...,"Tart and snappy, the flavors of lime flesh and...",87.0,14.0
3,3.0,US,Michigan,Lake Michigan Shore,Reserve Late Harvest,Riesling,St. Julian,2013.0,St. Julian 2013 Reserve Late Harvest Riesling ...,St. Julian 2013 Reserve Late Harvest Riesling ...,"Pineapple rind, lemon pith and orange blossom ...",87.0,13.0
4,4.0,US,Oregon,Willamette Valley,Vintner's Reserve Wild Child Block,Pinot Noir,Sweet Cheeks,2012.0,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,"Much like the regular bottling from 2012, this...",87.0,65.0


In [29]:
stemmer = nltk.stem.SnowballStemmer('english')

def stem_tokens(tokens, stemmer):
    stemmed = map(lambda x: stemmer.stem(x), tokens)
    return stemmed

def tokenize(text):
    lowers = text.lower()
    lowers=lowers.replace("-"," ")
    no_punctuation = lowers.translate(str.maketrans('','',string.punctuation))
    tokens = nltk.word_tokenize(no_punctuation)
    stems = stem_tokens(tokens, stemmer)
    return stems

### Calculate TF-IDF  and cosine similarity

#### Unigram TF-IDF
$$tf(t,d)=0.5+0.5\cdot \frac{f_{t,d}}{\max\{f_{t',d}:t'\in d\}},\  idf(t,D)=\log \frac{N}{|\{d\in D: t\in d\}|}$$   
$$tfidf(t,d,D)=tf(t,d)\cdot idf(t,D)$$  
- Indicate the importance of word in the wine description
- Unigram: single words (instead of characters as we did in text matching)  
    - We have also tried unigram and bigram models, but unigram model gives a more reasonalbe result.  
    - For example, the dictionary of unigram and bigram model contains both "Pinot" and "Pinot Noir", which produces worse performance for "Pinot Gris".  

- Choose the threshold for document frequency to select an appropriate dictionary size
    - Remove words with documnent frequency lower than 0.025 or higher than 0.95

In [32]:
tfidf = TfidfVectorizer(tokenizer=tokenize, stop_words='english',
                        ngram_range=(1,1), max_df=0.95, min_df=0.025,
                        use_idf=True, binary=False, norm=None)
tfidf_matrix = tfidf.fit_transform(wine['description'])
print(tfidf_matrix.shape)

(116759, 189)


Get a sparse TF-IDF matrix of dimension 116759×189, that is, the word dictionary has 189 words (stems).

In [33]:
print(tfidf.get_feature_names())

['accent', 'acid', 'add', 'age', 'alcohol', 'almond', 'alongsid', 'appl', 'apricot', 'aroma', 'aromat', 'attract', 'bake', 'balanc', 'barrel', 'beauti', 'berri', 'best', 'big', 'bit', 'bitter', 'black', 'blackberri', 'blend', 'blueberri', 'bodi', 'bottl', 'bright', 'bring', 'cabernet', 'candi', 'cassi', 'charact', 'chardonnay', 'cherri', 'chocol', 'cinnamon', 'citrus', 'clean', 'close', 'clove', 'coffe', 'cola', 'color', 'come', 'complex', 'concentr', 'core', 'creami', 'crisp', 'crush', 'currant', 'dark', 'delic', 'delici', 'deliv', 'dens', 'domin', 'dri', 'drink', 'earth', 'earthi', 'easi', 'edg', 'eleg', 'end', 'enjoy', 'feel', 'fine', 'finish', 'firm', 'flavor', 'floral', 'flower', 'follow', 'fresh', 'fruit', 'fruiti', 'generous', 'good', 'grape', 'grapefruit', 'great', 'green', 'herb', 'herbal', 'high', 'hint', 'honey', 'impress', 'intens', 'juici', 'just', 'layer', 'lead', 'leather', 'lemon', 'licoric', 'light', 'like', 'lime', 'linger', 'littl', 'live', 'long', 'make', 'matur', '

Given a specific wine, calculate the cosine similarity scores for all the other wines.  
For example, below is the recommendation results for wine of index 77662.

In [38]:
def get_recommendations(train, query):
    cos_query=cosine_similarity(train,query)
    cos_query=list(chain(*cos_query))
    sort_dist=np.argsort(cos_query)[::-1]
    return sort_dist[1:6].tolist()

In [39]:
recommend = [*map(lambda x: get_recommendations(tfidf_matrix,x),tfidf_matrix[77662])]
recommend

[[99224, 15491, 45437, 64740, 61945]]

Compare the wine descriptions

In [37]:
wine.loc[77662,'description']

"There's plenty of ripe pie-filling fruit flavor in this dry wine. Cherries, blackberries, boysenberries and all sorts of other red fruits and berries mingle with melted chocolate, licorice and peppery spice. Turns a bit hot on the finish."

In [64]:
wine.loc[99224,'description']

"This 2013 keeps up the full body and ripe fruit of previous releases, with lush texture and saturated chocolate and blackberry flavors. It's a dry red, however, with no detectable sweetness but plenty of tannin and satisfaction."

In [65]:
wine.loc[15491,'description']

"Spice, licorice and herbal notes complement red-fruit aromas, while the palate offers plenty of structure and tannic grab. There's an avalanche of blackberry, cassis, fig, chocolate and herbal flavors to process, while the finish is rich and rewarding before breaking up in bits."

In [66]:
wine.loc[45437,'description']

"This superrich Zin has extracted, luxurious fruit. It's ripe in blackberry and cherry jam, milk chocolate, peppery spice and anise flavors, with an overripe note of Port. Drink now."

In [67]:
wine.loc[64740,'description']

"Full, forward and loaded with ripe strawberry and cherry flavors, this immediately likeable wine offers plenty of fruit and spice. It's almost chunky it's so big, with a lick of chocolate highlighting the finish."

In [68]:
wine.loc[61945,'description']

"Stainless steel fermentation has yielded a yeasty, beery wine. There's plenty of fruit, a bit too reminiscent of canned peaches, and a finish that seems constricted and short. Residual sugar is right around 2%."